In [ ]:
%pip install git+https://github.com/suno-ai/bark.git
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-92st53qx
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-92st53qx
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download('punkt')

text="The church has a famous organ, built by Augustine Zeiger in 1841."
print(text)
sentences = nltk.sent_tokenize(text)
sentences

The church has a famous organ, built by Augustine Zeiger in 1841.


[nltk_data] Downloading package punkt to /home/pacotoh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The church has a famous organ, built by Augustine Zeiger in 1841.']

In [6]:
import numpy as np
from bark import generate_audio, SAMPLE_RATE
from transformers import BarkModel
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from IPython.display import Audio

GEN_TEMP = 0.6
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence
SPEAKER = "v2/en_speaker_9"
model = BarkModel.from_pretrained('suno/bark')

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )
    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

sampling_rate = model.generation_config.sample_rate
Audio(np.concatenate(pieces), rate=sampling_rate)

/var/data/python/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
100%|██████████| 17/17 [01:23<00:00,  4.94s/it]


## Simple preloaded models tests

In [1]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio

preload_models()

No GPU being used. Careful, inference might be very slow!
/var/data/python/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
text_prompt = """
     The church has a famous organ, built by Augustine Zeiger in 1841.
"""
audio_array = generate_audio(text_prompt)
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 15/15 [01:07<00:00,  4.52s/it]


In [7]:
text_prompt = """
     The church has a famous organ, built by Augustine Zeiger in 1841.
"""
audio_array = generate_audio(text_prompt, history_prompt="v2/en_speaker_9")
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 20/20 [01:35<00:00,  4.80s/it]
